# Data Loading

In [ ]:
import pandas as pd
DfBuyInfo = pd.read_csv('Datasets/INTERN_SVC_BUY.csv')
DfCustInfo = pd.read_csv('Datasets/INTERN_SVC_PRST_MONTH.csv')
DfSearchInfo = pd.read_csv('Datasets/INTERN_MENU_SRCH.csv')
#DfWatchInfo = pd.read_csv('Datasets/INTERN_SVC_VOD_WAT.csv',nrows=1000)

In [ ]:
# 시리즈별 시청 테이블 
DfWatchInfo = pd.read_csv(
    'Datasets/INTERN_SVC_VOD_WAT_SRIS_.csv',
    usecols = {
        'STRD_DT',
        'SVC_MGMT_NUM',
        'PROD_TYP_CD',
        'PAY_YN',
        'SRIS_ID',
        'SRIS_NM',
        'MV_OPEN_DT',
        'CTG_LVL1',
        'WAT_TMS_RNG_LST',
        'WAT_TMS',
        'WAT_CNT',
        'UNIQ_EPSD_CNT'
    },
    dtype = {
        'STRD_DT':'int64',
        'SVC_MGMT_NUM':'int64',
        'PROD_TYP_CD':'category',
        'BUY_NUM':'float64',
        'SALE_PRC':'float64',
        'PAY_YN':'category',
        'SRIS_ID':'category',
        'SRIS_NM':'object',
        'MV_OPEN_DT':'int64',
        'CTG_LVL1':'category',
        'WAT_TMS_RNG_LST':'object',
        'WAT_TMS':'float64',
        'WAT_CNT':'float64',
        'UNIQ_EPSD_CNT':'int64'
        },
    parse_dates=['STRD_DT']
) 

print(len(DfWatchInfo))
DfWatchInfo.head()

In [ ]:
a = pd.DataFrame(DfWatchInfo.groupby('SRIS_NM')['WAT_TMS'].mean())
a['WAT_TMS'].mean()/60

In [ ]:
DfBuyInfo.head()

In [ ]:
DfCustInfo.head()

In [ ]:
DfWatchInfo.head()

sec --> min 변환

In [ ]:
DfWatchInfo['WAT_MIN'] = DfWatchInfo['WAT_TMS'].apply(lambda x : round(int(x)/60,1))
DfWatchInfo.head()

# EDA

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
get_ipython().run_line_magic('matplotlib', 'inline')

#mpl.rc('font', family='NanumGothic')
#mpl.rc('axes', unicode_minus=False)

## 장르

### 장르 종류 및 장르별 시청 수

In [ ]:
CTGCount = pd.DataFrame(DfWatchInfo['CTG_LVL1'].value_counts().reset_index())
CTGCount.rename(columns = {'index':'CTG'},inplace=True)
CTGCount

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x='CTG',y='CTG_LVL1',data=CTGCount)
plt.title('CTG WAT count')
plt.xticks(rotation='45')
plt.show()

### 장르별 시청시간대 비교

In [ ]:
CTGWATRNGCount = pd.DataFrame(DfWatchInfo.groupby(['CTG_LVL1','WAT_TMS_RNG_CD'])['EPSD_NM'].count().reset_index())
CTGWATRNGCount = CTGWATRNGCount.sort_values(['CTG_LVL1','WAT_TMS_RNG_CD'],ascending=False)
#pd.set_option('display.max_rows', CTGWATRNGCount.shape[0]+1)
CTGWATRNGCount.head()

In [ ]:
plt.figure(figsize=(15,10))
sns.catplot(x = 'CTG_LVL1',y='EPSD_NM',hue='WAT_TMS_RNG_CD',data=CTGWATRNGCount,kind='bar',ax=ax)
plt.title('CTG WAT count')
plt.show()

### 장르별 시청시간 비교

In [ ]:
CTGWATTMSCount = pd.DataFrame(DfWatchInfo.groupby(['CTG_LVL1'])['WAT_MIN'].mean())
CTGWATTMSCount = CTGWATTMSCount.sort_values(['WAT_MIN'],ascending=False)
#pd.set_option('display.max_rows', CTGWATRNGCount.shape[0]+1
CTGWATTMSCount

In [ ]:
plt.figure(figsize=[15,10])
CTGWATTMSCount.plot(kind='bar')
plt.xticks(rotation='45')
plt.title('CTG WAT count')
plt.show()

### 시청시간대별 주 시청 장르

In [ ]:
WATRNGCTGCount = pd.DataFrame(DfWatchInfo.groupby(['WAT_TMS_RNG_CD', 'CTG_LVL1'])['EPSD_NM'].count().reset_index())
WATRNGCTGCount = WATRNGCTGCount.sort_values(['WAT_TMS_RNG_CD','EPSD_NM'],ascending=False)
#pd.set_option('display.max_rows', CTGWATRNGCount.shape[0]+1)
WATRNGCTGCount

## 시청 시간대별 시청 시간, 시청 수, 구매 수 (평균값, 한국영화)
<strong>유료와 무료로 나누어서 분석</strong>
유료에서는 PPM 30과 PPV 10 를 구분
- PPV + PAY NO = FOD
- PPM + PAY NO = 프로모션

In [ ]:
DfWatchInfo10 = DfWatchInfo[DfWatchInfo['PROD_TYP_CD']==10]
DfWatchInfo30 = DfWatchInfo[DfWatchInfo['PROD_TYP_CD']==30]

In [ ]:
DfKoreanMoviePayNo10 = DfWatchInfo10[(DfWatchInfo10['CTG_LVL1']=='1_한국영화')&(DfWatchInfo10['PAY_YN']=='N')]
tmp1 = pd.DataFrame(DfKoreanMoviePayNo10.groupby('WAT_TMS_RNG_CD')['WAT_MIN'].mean())
tmp2 = pd.DataFrame(DfKoreanMoviePayNo10.groupby('WAT_TMS_RNG_CD')['EPSD_NM'].count())
DfKoreanMoviePayNo10 = pd.concat([tmp1,tmp2],axis=1)

In [ ]:
DfKoreanMoviePayYes10 = DfWatchInfo10[(DfWatchInfo10['CTG_LVL1']=='1_한국영화')&(DfWatchInfo10['PAY_YN']=='Y')]
tmp1 = pd.DataFrame(DfKoreanMoviePayYes10.groupby('WAT_TMS_RNG_CD')['WAT_MIN'].mean())
tmp2 = pd.DataFrame(DfKoreanMoviePayYes10.groupby('WAT_TMS_RNG_CD')['EPSD_NM'].count())
DfKoreanMoviePayYes10 = pd.concat([tmp1,tmp2],axis=1)

In [ ]:
#현재 여건상 뽑아본 1000개 샘플중에는 ppm으로 한국영화를 보는 세톱박스가 없음
DfKoreanMoviePayYes30 = DfWatchInfo30[(DfWatchInfo30['CTG_LVL1']=='1_한국영화')&(DfWatchInfo30['PAY_YN']=='Y')]
tmp1 = pd.DataFrame(DfKoreanMoviePayYes30.groupby('WAT_TMS_RNG_CD')['WAT_MIN'].mean())
tmp2 = pd.DataFrame(DfKoreanMoviePayYes30.groupby('WAT_TMS_RNG_CD')['EPSD_NM'].count())
DfKoreanMoviePayYes30 = pd.concat([tmp1,tmp2],axis=1)

In [ ]:
# 프로모션 고객
#DfKoreanMoviePayNo30 = DfWatchInfo30[(DfWatchInfo30['CTG_LVL1']=='1_한국영화')&(DfWatchInfo30['PAY_YN']=='N')]
#tmp1 = pd.DataFrame(DfKoreanMoviePayNo30.groupby('WAT_TMS_RNG_CD')['WAT_MIN'].mean())
#tmp2 = pd.DataFrame(DfKoreanMoviePayNo30.groupby('WAT_TMS_RNG_CD')['EPSD_NM'].count())
#DfKoreanMoviePayNo30 = pd.concat([tmp1,tmp2],axis=1)

In [ ]:
DfKoreanMoviePayYes10.reset_index(inplace=True)
DfKoreanMoviePayYes10.rename(columns={'WAT_TMS_RNG_CD':'HOUR'},inplace=True)
DfKoreanMoviePayYes10.set_index('HOUR',inplace=True)
DfKoreanMoviePayYes10

- 구매 테이블에서 월정액은 에피소드 id가 남지 않음
    - 이유 : 구독권을 사는거지 단건 vod를 구매하는 것이 아니기 때문

In [ ]:
DfBuyInfo['HOUR'] = DfBuyInfo['BUY_DTM'].apply(lambda x: 'H'+str(x)[8:10]) #구매시간을 시간대별로 구분
DfBuyInfo10 = DfBuyInfo[DfBuyInfo['PROD_TYP_CD']==10]
DfBuyInfo10 = pd.merge(DfBuyInfo10[['STRD_DT','BUY_NUM','HOUR','CUST_NUM','SVC_MGMT_NUM','PROD_TYP_CD','SALE_PRC','EPSD_ID']],
                       DfWatchInfo[['EPSD_ID','CTG_LVL1']],on = 'EPSD_ID',how = 'inner' ) #장르 컬럼 붙이기
DfBuyInfo10 = DfBuyInfo10.drop_duplicates()
DfBuyInfo10.head()

In [ ]:
DfBuyInfoKoreaMv10 = DfBuyInfo10[DfBuyInfo10['CTG_LVL1']=='1_한국영화']
tmp = pd.DataFrame(DfBuyInfoKoreaMv10.groupby(['STRD_DT','HOUR'])['BUY_NUM'].count()).reset_index()
DfBuyInfoKoreaMv10 = pd.DataFrame(tmp.groupby('HOUR')['BUY_NUM'].mean())
DfBuyInfoKoreaMv10

merge DfKoreanMoviePayYes10 , DfBuyInfoKoreaMv10

In [ ]:
infoByHour = pd.concat([DfKoreanMoviePayYes10,DfBuyInfoKoreaMv10],axis=1)
infoByHour.columns = ['PAY_YES_WAT_MIN','PAY_YES_WAT_COUNT','BUY_COUNT']
infoByHour.sort_values('PAY_YES_WAT_MIN',ascending=False)
infoByHour.reset_index(inplace=True)
infoByHour.rename(columns={'index':'HOUR'},inplace=True)
infoByHour.head()

ppv+ 무료 = fod의 시간대별 시청패턴

In [ ]:
DfKoreanMoviePayNo10.reset_index(inplace=True)
DfKoreanMoviePayNo10.rename(columns={'WAT_TMS_RNG_CD':'HOUR','EPSD_NM':'PAY_NO_WAT_COUNT','WAT_MIN':'PAY_NO_WAT_MIN'},inplace=True)
DfKoreanMoviePayNo10.sort_values('PAY_NO_WAT_MIN',ascending=False)

* <STRONG>BUY_COUNT는 PAY_YES에 한해서 COUNT</STRONG>

In [ ]:
TotalInfoByHour = pd.merge(infoByHour,DfKoreanMoviePayNo10, on ='HOUR',how='inner')
TotalInfoByHour.set_index('HOUR',inplace=True)
TotalInfoByHour.sort_values(['PAY_YES_WAT_MIN'],ascending=False)

## 유료 vs 무료

### 장르별 시청수


In [ ]:
PAYYNCTGCount = pd.DataFrame(DfWatchInfo.groupby(['PAY_YN','CTG_LVL1'])['EPSD_NM'].count().reset_index())
PAYYNCTGCount = PAYYNCTGCount.sort_values(['PAY_YN','EPSD_NM'],ascending=False)
PAYYNCTGCount

In [ ]:
plt.figure(figsize=(15,10))
sns.catplot(x = 'CTG_LVL1',y='EPSD_NM',hue='PAY_YN',data=PAYYNCTGCount,kind='bar')
plt.title('PAY_YN CTG count')
plt.xticks(rotation='45')
plt.show()

### 장르별 시청 시간

In [ ]:
PAYYNCTGTMSMean = pd.DataFrame(DfWatchInfo.groupby(['PAY_YN','CTG_LVL1'])['WAT_MIN'].mean().reset_index())
PAYYNCTGTMSMean = PAYYNCTGTMSMean.sort_values(['PAY_YN','WAT_MIN'],ascending=False)
PAYYNCTGTMSMean

In [ ]:
plt.figure(figsize=(15,10))
sns.catplot(x = 'CTG_LVL1',y='WAT_MIN',hue='PAY_YN',data=PAYYNCTGTMSMean,kind='bar')
plt.title('WAT MIN mean')
plt.xticks(rotation='45')
plt.show()

### 장르별 시청시간 분포

pay_yn : N, CTG : 

In [ ]:
ctgList = DfWatchInfo[DfWatchInfo['PAY_YN']=='N']['CTG_LVL1'].unique().tolist()
print(len(ctgList))

In [ ]:
nPayCtgWatchDist = DfWatchInfo[DfWatchInfo['PAY_YN']=='N']
yPayCtgWatchDist = DfWatchInfo[DfWatchInfo['PAY_YN']=='Y']


In [ ]:
fig, axes = plt.subplots(nrows=8, ncols=2,figsize=(15,30))
for i, ctg in enumerate(ctgList):
    sns.distplot(nPayCtgWatchDist[nPayCtgWatchDist['CTG_LVL1']==ctg]['WAT_MIN'].values,ax=axes[i][0])
    sns.distplot(yPayCtgWatchDist[yPayCtgWatchDist['CTG_LVL1']==ctg]['WAT_MIN'].values,ax=axes[i][1])
    axes[i][0].set_title(ctg+'PAY NO mean watch MIN')
    axes[i][1].set_title(ctg+'PAY YES mean watch MIN')
plt.show()

## 연령별 분석 (한국영화, 해외 영화)

### 주 나이대

### 주 시청 시간대

### 시청 시간 추이

## 시청 건수 중에서 검색 진입 비중 파악

In [ ]:
print(len(DfSearchInfo))
DfSearchInfo.head()

In [ ]:
print(len(DfWatchInfo))
DfWatchInfo.head()

In [ ]:
mergeWatch = DfWatchInfo[['SRIS_ID','CTG_LVL1']].drop_duplicates()
searchMerged = pd.merge(DfSearchInfo, mergeWatch, on = 'SRIS_ID', how ='inner')
searchMerged.head()

### 카테고리별 월별 검색 수

In [ ]:
searchMerged['MONTH'] = searchMerged['STRD_DT'].apply(lambda x :str(x)[:-2]).astype(int)
searchMerged.head()

In [ ]:
searchGroupCnt = searchMerged.groupby(['MONTH','CTG_LVL1'])['PATH_GNB_MENU_NM'].count()
searchGroupCnt = pd.DataFrame(searchGroupCnt).rename(columns = {'PATH_GNB_MENU_NM': 'SEARCH_CNT'})
searchGroupCnt

### 카테고리별 월별 시청 수 

In [ ]:
DfWatchInfo['MONTH'] = DfWatchInfo['STRD_DT'].apply(lambda x :str(x)[:-2]).astype(int)
DfWatchInfo.head()